<a href="https://colab.research.google.com/github/saksham751/Neural-Network-from-Scratch/blob/some-QOL-changes/neural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [117]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets.mnist import load_data
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Neural Network Class

In [118]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None
    def forward_propagation(self, input):
        raise NotImplementedError
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

In [119]:
class completeLayer(Layer):
  
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size)
        self.bias = np.random.rand(1, output_size)

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        #print(output_error.shape, self.weights.T.shape)
        #print(self.input.T.shape, output_error.shape)
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T.reshape(self.input.T.shape[0],1), output_error)
        
        self.weights -= learning_rate * weights_error
        self.bias = self.bias-learning_rate * output_error
        return input_error

In [120]:

class activationFunction(Layer):
    def __init__(self, activation, activationDer):
        self.activation = activation
        self.activationDer = activationDer

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        return self.activationDer(self.input) * output_error

In [121]:
    def sig(x):
      return 1/(1 + np.exp(-x))

    def sig_der(x):
      return sig(x)*(1-sig(x))
  
    def tanh(x):
      return np.tanh(x)

    def tanh_prime(x):
      return 1-np.tanh(x)**2

    def relu(x):
      return np.maximum(x, 0)

    def relu_der(x):
      return 1. * (x > 0)

    def linear(x):
      return x

    def linear_der(x):
      return 1

    

In [122]:

def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mseDer(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

In [123]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    def add(self, layer):
        self.layers.append(layer)

    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    def predict(self, input_data):

        samples = len(input_data)
        result = []
        for i in range(samples):

            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    def fit(self, x_train, y_train, epochs, learning_rate):

        samples = len(x_train)
        for i in range(epochs):
            err = 0
            for j in range(samples):
  
                output = x_train[j]
                
                for layer in self.layers:
                    output = layer.forward_propagation(output)
                err += self.loss(y_train[j], output)

                error = self.loss_prime(y_train[j], output)
                #error.reshape(784,1)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)
                #print('error=%f'%(err/j))

            err /= samples
            if((i+1)%10==0 or i==0):
              print('epoch %d/%d   error=%f' % (i+1, epochs, err))

In [124]:
def classAccuracy( y_test,res):
  cor=0
  for x,y in zip(y_test,res):
    cor+=x==y
  acc=cor/len(res)*100
  return acc

In [125]:
def mseRegression(y_test,res):
  errors = mean_squared_error(y_test, res)
  return errors

# Classification Task

In [327]:
#Loading the iris dataset
iris = datasets.load_iris()
x, y = iris.data, iris.target

In [328]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=123)

In [329]:
#calculating the number of classes in the dataset
x_train.shape[1]
classes=np.unique(y_train).size
y_train

array([2, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 0, 2, 2, 2, 2, 2, 1, 0, 0, 2, 0,
       0, 1, 1, 1, 1, 2, 1, 2, 0, 2, 1, 0, 0, 2, 1, 2, 2, 0, 1, 1, 2, 0,
       2, 1, 1, 0, 2, 2, 0, 0, 1, 1, 2, 0, 0, 1, 0, 1, 2, 0, 2, 0, 0, 1,
       0, 0, 1, 2, 1, 1, 1, 0, 0, 1, 2, 0, 0, 1, 1, 1, 2, 1, 1, 1, 2, 0,
       0, 1, 2, 2, 2, 2, 0, 1, 0, 1, 1, 0, 1, 2, 1, 2, 2, 0, 1, 0, 2, 2,
       1, 1, 2, 2, 1, 0, 1, 1, 2, 2])

In [330]:
#converting to binary encoded categorical values
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train)
y_train.shape

(120, 3)

In [331]:
network1 = Network()
x1=x_train.shape[1]
x2=4
network1.add(completeLayer(x1, x2))
network1.add(activationFunction(tanh, tanh_prime))
network1.add(completeLayer(x2, classes))
network1.add(activationFunction(tanh, tanh_prime))


In [332]:
x_train.shape

(120, 4)

In [333]:
network1.use(mse, mseDer)

network1.fit(x_train ,y_train, epochs=5000, learning_rate=0.0005)

epoch 1/5000   error=0.649241
epoch 10/5000   error=0.647529
epoch 20/5000   error=0.645170
epoch 30/5000   error=0.642126
epoch 40/5000   error=0.638048
epoch 50/5000   error=0.632300
epoch 60/5000   error=0.623626
epoch 70/5000   error=0.609275
epoch 80/5000   error=0.582956
epoch 90/5000   error=0.536655
epoch 100/5000   error=0.495633
epoch 110/5000   error=0.480198
epoch 120/5000   error=0.464439
epoch 130/5000   error=0.436731
epoch 140/5000   error=0.400225
epoch 150/5000   error=0.383251
epoch 160/5000   error=0.381030
epoch 170/5000   error=0.380871
epoch 180/5000   error=0.380857
epoch 190/5000   error=0.380854
epoch 200/5000   error=0.380852
epoch 210/5000   error=0.380850
epoch 220/5000   error=0.380847
epoch 230/5000   error=0.380845
epoch 240/5000   error=0.380843
epoch 250/5000   error=0.380841
epoch 260/5000   error=0.380839
epoch 270/5000   error=0.380837
epoch 280/5000   error=0.380835
epoch 290/5000   error=0.380833
epoch 300/5000   error=0.380831
epoch 310/5000   er

In [334]:
x_test

array([[6.3, 2.5, 4.9, 1.5],
       [6.8, 3. , 5.5, 2.1],
       [6.4, 2.8, 5.6, 2.2],
       [5.6, 3. , 4.1, 1.3],
       [4.9, 3.6, 1.4, 0.1],
       [6. , 3. , 4.8, 1.8],
       [6.3, 2.3, 4.4, 1.3],
       [4.4, 3.2, 1.3, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [5.5, 2.6, 4.4, 1.2],
       [6.9, 3.1, 5.1, 2.3],
       [5.5, 4.2, 1.4, 0.2],
       [5.2, 2.7, 3.9, 1.4],
       [6.5, 3. , 5.5, 1.8],
       [7.7, 3. , 6.1, 2.3],
       [6.5, 3. , 5.8, 2.2],
       [5.5, 3.5, 1.3, 0.2],
       [4.3, 3. , 1.1, 0.1],
       [6.1, 2.9, 4.7, 1.4],
       [4.8, 3. , 1.4, 0.3],
       [5.2, 3.4, 1.4, 0.2],
       [6.3, 2.8, 5.1, 1.5],
       [4.8, 3.4, 1.9, 0.2],
       [6.1, 3. , 4.9, 1.8],
       [5.1, 3.8, 1.6, 0.2],
       [5.4, 3.4, 1.7, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.6, 2.8, 4.9, 2. ],
       [7.7, 3.8, 6.7, 2.2],
       [5. , 3.6, 1.4, 0.2]])

In [335]:
out = network1.predict(x_test)
out=[x[0] for x in out]

In [336]:
out

[array([0.18301342, 0.38488205, 0.48226518]),
 array([-0.00172952,  0.41032279,  0.65210075]),
 array([-0.07639578,  0.42033712,  0.70655149]),
 array([0.14733855, 0.38992516, 0.51946369]),
 array([ 0.62793219,  0.30619936, -0.2192452 ]),
 array([0.00399594, 0.40955235, 0.64762146]),
 array([0.36299249, 0.35765806, 0.25573629]),
 array([ 0.58695173,  0.31570387, -0.13408475]),
 array([ 0.59591022,  0.31369385, -0.15238974]),
 array([0.14165877, 0.3907207 , 0.52517547]),
 array([0.02236946, 0.40707519, 0.63293872]),
 array([ 0.64726268,  0.30141261, -0.26079048]),
 array([0.09308066, 0.39745262, 0.57179059]),
 array([0.00259455, 0.40974094, 0.64872208]),
 array([0.0471562 , 0.40372042, 0.61236344]),
 array([-0.09298982,  0.42256058,  0.71770667]),
 array([ 0.67476183,  0.29420557, -0.32133999]),
 array([ 0.61717611,  0.30877873, -0.19660367]),
 array([0.14831161, 0.38978846, 0.5184799 ]),
 array([ 0.62434413,  0.30705953, -0.21165064]),
 array([ 0.65319409,  0.29989856, -0.27373332]),
 

In [337]:
res=np.argmax(out,axis=1)
res

array([2, 2, 2, 2, 0, 2, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 0, 0, 2, 0, 0, 2,
       0, 2, 0, 0, 0, 2, 2, 0])

In [338]:
y_test

array([1, 2, 2, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 2, 2, 2, 0, 0, 1, 0, 0, 2,
       0, 2, 0, 0, 0, 2, 2, 0])

In [339]:
classAccuracy(y_test,res)

80.0

# Regression Task

In [256]:
#loading the diabetes dataset
db = datasets.load_diabetes()
x, y = db.data, db.target

In [257]:
#SPlitting data for training and testing
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=123)

In [258]:
x_train
y_train

array([170.,  77.,  49.,  66., 144., 113., 233., 162., 121.,  88., 302.,
       128.,  52., 178.,  90., 164., 135., 103., 200., 178., 261.,  64.,
        59.,  79.,  47., 107.,  39., 151., 104., 217., 232.,  55., 245.,
       131.,  64., 222., 249., 128., 293., 138., 273., 158., 168., 103.,
       140., 146., 277., 116., 233.,  94., 295., 191., 241., 154., 141.,
       252., 198., 288., 101., 179., 129., 174.,  91.,  84., 177., 219.,
        67., 113.,  70.,  53., 150.,  40., 258., 168., 283., 252.,  43.,
        74.,  78., 209., 270.,  90.,  73., 310., 102., 115.,  85., 104.,
        68., 101., 139.,  63.,  72.,  58., 131.,  71.,  98.,  88., 277.,
       104., 167., 108., 111., 214., 135., 196., 189.,  77., 225.,  60.,
        67., 292., 143., 281.,  25., 200.,  91., 263.,  95., 144., 232.,
       109.,  51.,  64., 283., 259., 214.,  60., 111., 258.,  68., 174.,
       118., 113., 306.,  70.,  78., 191., 220., 183., 124.,  44.,  63.,
       114., 229.,  77., 168., 262.,  91.,  85., 12

In [259]:
network2 = Network()
x1=x_train.shape[1]
x2=x1*2
network2.add(completeLayer(x1, x2))
network2.add(activationFunction(relu, relu_der))
network2.add(completeLayer(x2, 1))
network2.add(activationFunction(linear, linear_der))


In [263]:
network2.use(mse, mseDer)

network2.fit(x_train ,y_train, epochs=2000, learning_rate=0.0001)

epoch 1/2000   error=2593.215550
epoch 10/2000   error=2585.068773
epoch 20/2000   error=2584.030870
epoch 30/2000   error=2582.183406
epoch 40/2000   error=2581.992855
epoch 50/2000   error=2580.140516
epoch 60/2000   error=2579.731850
epoch 70/2000   error=2579.869824
epoch 80/2000   error=2579.406875
epoch 90/2000   error=2578.774139
epoch 100/2000   error=2578.124666
epoch 110/2000   error=2577.023827
epoch 120/2000   error=2577.641202
epoch 130/2000   error=2575.459268
epoch 140/2000   error=2576.945679
epoch 150/2000   error=2578.089902
epoch 160/2000   error=2577.846628
epoch 170/2000   error=2576.950619
epoch 180/2000   error=2573.949111
epoch 190/2000   error=2574.594181
epoch 200/2000   error=2572.846671
epoch 210/2000   error=2573.060157
epoch 220/2000   error=2573.554423
epoch 230/2000   error=2574.095705
epoch 240/2000   error=2575.505535
epoch 250/2000   error=2573.127460
epoch 260/2000   error=2572.557954
epoch 270/2000   error=2571.978037
epoch 280/2000   error=2571.679

In [264]:
out = network2.predict(x_test)
out=[x[0] for x in out]
print(out)

[array([141.53651924]), array([103.91515197]), array([199.66457149]), array([146.44360385]), array([162.98055996]), array([159.94605977]), array([273.17201149]), array([101.72440513]), array([115.01097642]), array([113.93335237]), array([141.51252022]), array([170.98917301]), array([149.95128532]), array([175.90974391]), array([212.42078086]), array([170.49729931]), array([107.1215083]), array([140.43178118]), array([139.95313456]), array([230.9493396]), array([157.5935252]), array([251.41743769]), array([119.94736336]), array([60.2553187]), array([94.34941294]), array([169.04526002]), array([74.70387821]), array([96.72373069]), array([147.27410739]), array([161.06877833]), array([93.41353303]), array([223.33280547]), array([191.79636805]), array([221.60925912]), array([174.87387567]), array([65.19699978]), array([68.67209367]), array([112.25474103]), array([218.27341893]), array([81.74236011]), array([198.53604467]), array([69.8977308]), array([113.93465012]), array([120.44347312]), a

In [265]:
mseRegression(y_test,out)

2660.406687785831